In [1]:
import pickle, yaml, os
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from utils import (
    disagg_2_agg_trip,
    get_rider,
)


In [2]:
with open("config_mc_m2m_3.yaml", "r+") as fopen:
    config = yaml.load(fopen, Loader=yaml.FullLoader)
with open("config_DaR.yaml", "r+") as fopen:
    config_DaR = yaml.load(fopen, Loader=yaml.FullLoader)

figure_pth = config["figure_pth"]
figure_pth_DaR = config_DaR["figure_pth"]
if not os.path.exists(figure_pth):
    os.makedirs(figure_pth)
if not os.path.exists(figure_pth_DaR):
    os.makedirs(figure_pth_DaR)


In [3]:
Y_df = pd.read_csv(config["m2m_output_loc"] + "Y_rdl_disagg.csv")
Y_df = Y_df.iloc[:-1, :].astype("int64")
Y = list(Y_df.to_records(index=False))
R = set(Y_df.r)
r_transfer = {}

Y_df_DaR = pd.read_csv(config_DaR["m2m_output_loc"] + "Y_rdl_disagg.csv")
Y_df_DaR = Y_df_DaR.iloc[:-1, :].astype("int64")
Y_DaR = list(Y_df_DaR.to_records(index=False))
R_DaR = set(Y_df_DaR.r)
r_transfer_DaR = {}


In [4]:
dumm_d = (
    max(config["driver_set"]) + 1 if config["driver_set"] else len(config["FR_list"])
)

for (r, d) in zip(Y_df.r, Y_df.d):
    if d != dumm_d:
        r_transfer.setdefault(r, set()).add(d)

n_transfer = {}
for r, D in r_transfer.items():
    n_transfer.setdefault(len(D) - 1, 0)
    n_transfer[len(D) - 1] += 1

n_transfer = {k: v / len(R) for (k, v) in n_transfer.items()}


In [5]:
dumm_d_DaR = 1

for (r, d) in zip(Y_df_DaR.r, Y_df_DaR.d):
    if d != dumm_d:
        r_transfer_DaR.setdefault(r, set()).add(d)

n_transfer_DaR = {}
for r, D in r_transfer_DaR.items():
    n_transfer_DaR.setdefault(len(D) - 1, 0)
    n_transfer_DaR[len(D) - 1] += 1

n_transfer_DaR = {k: v / len(R) for (k, v) in n_transfer_DaR.items()}


In [6]:
ttrips_mat = pickle.load(
    open(config["m2m_data_loc"] + "ttrips_mat.p", "rb"),
)

# disaggregate network, dis_2_agg is 1-to-1 mapping
disagg_2_agg_id = {k: k for k in range(config["S_disagg"])}
trip_dict = disagg_2_agg_trip(
    ttrips_mat, config, disagg_2_agg_id=disagg_2_agg_id, fraction=3 / 24
)
Rider = get_rider(trip_dict, config)

Rider_DaR = pd.read_csv(config_DaR["m2m_data_loc"] + "Rider_agg.csv").to_numpy()


In [7]:
on_time, travel_time, wait_stop, wait_time = dict(), dict(), dict(), defaultdict(int)
for i, r, d, t1, t2, s1, s2 in zip(
    Y_df.index, Y_df.r, Y_df.d, Y_df.t1, Y_df.t2, Y_df.s1, Y_df.s2
):
    if s1 == Rider[r][4] and r not in on_time:
        on_time[r] = t1
    if s2 == Rider[r][5]:
        travel_time[r] = t2 - on_time[r]
    if i == 0:
        if d == dumm_d:
            wait_stop[r] = (t1, Y_df.loc[i - 1, "d"])
    else:
        if d == dumm_d and (
            Y_df.loc[i - 1, "r"] != r or Y_df.loc[i - 1, "d"] != dumm_d
        ):
            wait_stop[r] = (t1, Y_df.loc[i - 1, "d"])
        elif (
            (Y_df.loc[i - 1, "r"], Y_df.loc[i - 1, "d"]) == (r, dumm_d)
            and d != dumm_d
            and d != wait_stop[r][1]
        ):
            wait_time[r] += t1 - wait_stop[r][0]


In [8]:
print(
    "Average transfer waiting time without DaR: {}".format(
        np.mean(list(wait_time.values())) if wait_time else 0
    )
)
print("Average travel time without DaR: {}".format(np.mean(list(travel_time.values()))))


Average transfer waiting time without DaR: 3.0
Average travel time without DaR: 12.458333333333334


In [8]:
N_r = len(Rider)
for i, r, d, t1, t2, s1, s2 in zip(
    Y_df_DaR.index,
    Y_df_DaR.r,
    Y_df_DaR.d,
    Y_df_DaR.t1,
    Y_df_DaR.t2,
    Y_df_DaR.s1,
    Y_df_DaR.s2,
):
    if s1 == Rider_DaR[r][4] and r + N_r not in on_time:
        on_time[r + N_r] = t1
    if s2 == Rider_DaR[r][5]:
        travel_time[r + N_r] = t2 - on_time[r + N_r]
        travel_time[r + 2 * N_r] = t2 - on_time[r + N_r]
        travel_time[r + 3 * N_r] = t2 - on_time[r + N_r]
    if i == 0:
        if d == dumm_d_DaR:
            wait_stop[r + N_r] = (t1, Y_df_DaR.loc[i - 1, "d"])
            wait_stop[r + 2 * N_r] = (t1, Y_df_DaR.loc[i - 1, "d"])
            wait_stop[r + 3 * N_r] = (t1, Y_df_DaR.loc[i - 1, "d"])
    else:
        if d == dumm_d_DaR and (
            Y_df_DaR.loc[i - 1, "r"] != r or Y_df_DaR.loc[i - 1, "d"] != dumm_d_DaR
        ):
            wait_stop[r + N_r] = (t1, Y_df_DaR.loc[i - 1, "d"])
            wait_stop[r + 2 * N_r] = (t1, Y_df_DaR.loc[i - 1, "d"])
            wait_stop[r + 3 * N_r] = (t1, Y_df_DaR.loc[i - 1, "d"])
        elif (
            (Y_df_DaR.loc[i - 1, "r"], Y_df_DaR.loc[i - 1, "d"]) == (r, dumm_d_DaR)
            and d != dumm_d_DaR
            and d != wait_stop[r + N_r][1]
        ):
            wait_time.setdefault(r + N_r, 0)
            wait_time.setdefault(r + 2 * N_r, 0)
            wait_time.setdefault(r + 3 * N_r, 0)
            wait_time[r + N_r] += t1 - wait_stop[r + N_r][0]
            wait_time[r + 2 * N_r] += t1 - wait_stop[r + N_r][0]
            wait_time[r + 3 * N_r] += t1 - wait_stop[r + N_r][0]


In [9]:
print(
    "Average transfer waiting time with DaR: {}".format(
        np.mean(list(wait_time.values())) if wait_time else 0
    )
)
print("Average travel time with DaR: {}".format(np.mean(list(travel_time.values()))))


Average transfer waiting time: 0
Average travel time: 10.86231884057971


In [11]:
Y_df


,r,d,t1,t2,s1,s2,n1,n2
0,0,1,0,5,1,39,1,384
1,1,0,35,40,1,39,2416,2799
2,2,0,35,40,1,39,2416,2799
3,3,0,35,40,1,39,2416,2799
4,4,0,35,40,1,39,2416,2799
...,...,...,...,...,...,...,...,...
384,278,4,19,20,30,30,1341,1410
385,278,4,20,21,30,30,1410,1479
386,278,4,21,22,30,30,1479,1548
387,278,4,22,23,30,30,1548,1617
